In [17]:
import glob
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
from IPython.core.debugger import set_trace
import difflib
import csv

In [2]:
# load data

alldata_new = pickle.load(open('output/dtm_processed_output.p', 'rb'))
alldata_new.keys()
doc_year=alldata_new['docs_per_year']
doc_ids =[0]+list(np.cumsum(doc_year))

term_topic = alldata_new['term_topic']# term_topic is n_years*n_topics*n_terms
terms = alldata_new['terms']
term_frequency = alldata_new['term_frequency'][1:] # weirdly the first entry is empty
doc_topicyrs = alldata_new['doc_topic']

doc_topic = []
doc_length=[]
for year in range(len(term_topic)):    
    doc_topic.append(alldata_new['doc_topic'][doc_ids[year]:doc_ids[year+1]])# doc_topic is nyear*n_docs given year*n_topics
    doc_length.append(alldata_new['doc_length'][doc_ids[year]:doc_ids[year+1]]) #doc_length is nyear*n_docs given year"""    
# rename topics by the hand-picked names
topic_labels = pickle.load(open('topicnames.p','rb'))
    

In [3]:
def stringdiff(a,b):
    return sum ( a[i] != b[i] for i in range(len(a)) )

In [4]:
def finddoc(alldocs,title):
    for doc in alldocs:
        """print(doc)         
        matchratio = difflib.SequenceMatcher(None,title,doc).ratio()
        print(matchratio)
        if matchratio >.9:
            print(doc+'\n'+title)
            idx = alldocs.index(doc)
            return([year,idx])"""
        if len(doc)==len(title):
            if stringdiff(doc,title)<3:
                #print(doc+'\n'+title)
                idx = alldocs.index(doc)
                return([year,idx])
    return([])
                


In [15]:
def getlist(titles):
    titles = [k.lower() for k in titles]
    doclist=[]
    titlelist=[]
    for t in titles:
        didfind=False
        for year in range(18):
            docdir = 'text_data/volume_{}/'.format(22+year)
            alldocs = glob.glob(docdir+'*.txt')
            alldocs = [k.lower()[20:-4] for k in alldocs]
            find=finddoc(alldocs,t)
            if find:
                doclist.append(find)
                titlelist.append(t)
                didfind=True
                break

        if didfind==False:
            print('\ncannot find: '+t)
    return([doclist,titlelist])
# given a list of paper, what are their main topics? for analyzing like a lab or an author
def maintopics(doclist,*topic_labels):
    ntopics=20
    doc_topfreq=np.empty((len(doclist),ntopics))
    for k in range(len(doclist)):
        if len(doclist[0])==1:
            doc_topfreq[k]=alldata_new['doc_topic'][doclist[k]]
        elif len(doclist[0])==2: # year then index
            doc_topfreq[k]=doc_topic[doclist[k][0]][doclist[k][1]]
            
    doc_topfreq = np.mean(doc_topfreq,axis=0)
    doc_topfreq = doc_topfreq/sum(doc_topfreq)
    print(doc_topfreq)
    maintopid = np.argsort(-doc_topfreq) 
    doc_topfreq=doc_topfreq[maintopid]
    if topic_labels:
        maintopics=[topic_labels[0][idx] for idx in maintopid]
    return (maintopics,doc_topfreq)


In [20]:
label='alex'
titles=['Computationally reproducible experiments','The Attentional Learning Trap and How to Avoid it','Online Experiments using jsPsych, psiTurk, and Amazon Mechanical Turk']
[doclist,titlelist]=getlist(titles)        
(mtops,meantpfreq)=maintopics(doclist,topic_labels)
with open('result/lab_topic/'+label+'.txt','w') as f:
    for k in range(len(mtops)):
        f.write(mtops[k]+', freq={}'.format(meantpfreq[k])+'\n')
    f.write('\n papers included:\n')
    for title in titlelist:
        f.write(title)
        
with open('result/lab_topic/'+label+'.csv','w') as f:
    csvwriter = csv.writer(f)
    for k in range(len(mtops)):
        csvwriter.writerow([mtops[k],meantpfreq[k]])


cannot find: computationally reproducible experiments

cannot find: online experiments using jspsych, psiturk, and amazon mechanical turk
[  3.52360824e-05   3.52360824e-05   2.83207576e-02   3.52360826e-05
   9.24625903e-03   3.52360824e-05   3.52360823e-05   3.52360825e-05
   3.52360827e-05   1.74083251e-02   2.69273092e-03   3.52368036e-05
   3.52360823e-05   3.52360822e-05   3.52360825e-05   3.89713131e-01
   3.52360823e-05   5.52125491e-01   3.52360827e-05   3.52360824e-05]


In [11]:
label='anselm'
titles=['Asking and evaluating natural language questions']
[doclist,titlelist]=getlist(titles)        
(mtops,meantpfreq)=maintopics(doclist,topic_labels)
with open('result/lab_topic/'+label+'.txt','w') as f:
    for k in range(len(mtops)):
        f.write(mtops[k]+', freq={}'.format(meantpfreq[k])+'\n')
    f.write('\n papers included:\n')
    for title in titlelist:
        f.write(title)

[  3.12893817e-05   8.85112609e-03   3.12891121e-05   1.51686260e-02
   1.23247133e-02   7.62020917e-02   3.12891121e-05   3.12891121e-05
   3.55792874e-01   3.12891119e-05   1.72890294e-01   6.94449590e-02
   3.12891121e-05   5.06132765e-02   2.38367859e-01   3.12891120e-05
   3.12891119e-05   3.12891121e-05   3.12891121e-05   3.12891121e-05]
> <ipython-input-9-c71673ab08b7>(36)maintopics()
     34     doc_topfreq=doc_topfreq[maintopid]
     35     set_trace()
---> 36     if topic_labels:
     37         maintopics=[topic_labels[0][idx] for idx in maintopid]
     38     return (maintopics,doc_topfreq)

ipdb> c


In [16]:
# find the paper index for given titles
label='gureckis'
titles = ['Information selection in categorization with stimulus uncertainty','Beliefs about sparsity affect casual experimentation',
         'Asking and evaluating natural language questions','The distorting effects of deciding to stop sampling',
         'Desirable difficulties in the development of active inquiry skills',
         'Active control of study leads to improved recognition memory in children',
          'Deep Neural Networks Predict Category Typicality Ratings for Images',
         'the Attentional Learning Trap and How to Avoid it',
         'Are Biases When Making Causal Interventions Related to Biases in Belief Updating',
         'Decisions to intervene on causal systems are adaptively selected',
         'The value of approaching bad things',
         'A preference for the unpredictable over the informative during self-directed learning',
         'Workshop: Online Experiments using jsPsych, psiTurk, and Amazon Mechanical Turk',
         'Informavores: Active information foraging and human cognition',
         'Does the utility of information influence sampling behavior',
         'One piece at a time: Learning complex rules through self-directed sampling',
         'Sparse category labels obstruct generalization of category membership',
         'One-shot lotteries in the park',
         'Does category labeling lead to forgetting',
         "Don't Stop 'Til You Get Enough: Adaptive Information Sampling in a Visuomotor Estimation Task",
         'Grow your own representations: Computational constructivism',
         'Category Learning Through Active Sampling',
         'The Impact of Perceptual Aliasing on Exploration and Learning in a Dynamic Decision Making Task',
         'Is categorial perception really verbally-mediated perception',
         'Active Learning Strategies in a Spatial Concept Learning Game',
         'When Things Get Worse before they Get Better: Regulatory Fit and Average-Reward Learning in a Dynamic Decision-Making Environment',
         'The emergence of Collective Structure through Individual Interactions',
         'Modeling category intuitiveness',
         'The effect of the internal structure of categories on perception']            
[doclist,titlelist]=getlist(titles)        
(mtops,meantpfreq)=maintopics(doclist,topic_labels)
with open('result/lab_topic/'+label+'.txt','w') as f:
    for k in range(len(mtops)):
        f.write(mtops[k]+', freq={}'.format(meantpfreq[k])+'\n')
    f.write('\n papers included:\n')
    for title in titlelist:
        f.write(title)


cannot find: information selection in categorization with stimulus uncertainty

cannot find: the distorting effects of deciding to stop sampling

cannot find: active control of study leads to improved recognition memory in children

cannot find: workshop: online experiments using jspsych, psiturk, and amazon mechanical turk

cannot find: informavores: active information foraging and human cognition

cannot find: one piece at a time: learning complex rules through self-directed sampling

cannot find: don't stop 'til you get enough: adaptive information sampling in a visuomotor estimation task

cannot find: grow your own representations: computational constructivism

cannot find: is categorial perception really verbally-mediated perception

cannot find: when things get worse before they get better: regulatory fit and average-reward learning in a dynamic decision-making environment

cannot find: the emergence of collective structure through individual interactions

cannot find: the effec

In [72]:
print(topic_labels)

['Sequential learning', 'Face and emotion perception', 'Reasoning', 'Text processing and creativity', 'Mathematical cognition', 'Decision making', 'Language: syntax', 'Causal reasoning', 'Knowledge structure', 'Developmental psychology', 'Spatial cognition and embodied cognition', 'Memory', 'Categorization', 'Language: semantics', 'Educational psychology', 'Network models for cognition and Neuroscience', 'non-verbal communication', 'Probabilistic modeling', 'Consciousness and identity', 'Visual attention']


In [8]:
meantpfreq

array([ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan])